In [1]:
import faiss
import pandas as pd

paths = {
    "index": "vector.index",
    "df": "dataframe.pkl"
}
df = pd.read_pickle(paths["df"])
index = faiss.read_index(paths["index"], faiss.IO_FLAG_MMAP | faiss.IO_FLAG_READ_ONLY)

In [2]:
import sys
sys.path.append("../")
from models.sbert import load
sbert = load()

In [3]:
df.head()

,id,para_id,sent_id,num_nps,num_ents,sent_text,sbert_768
0,tagd-2021-84238_0_0,[0],[0],2,0,Skattemyndighetene har over tid ført kontroll ...,"[0.0014578121481463313, -0.41081535816192627, ..."
1,tagd-2021-84238_0_1,[0],[1],3,2,"Det er D (tidligere D), sammen med E, som oppl...","[1.3350529670715332, -0.050390999764204025, 1...."
2,tagd-2021-84238_1_0,[1],[0],6,4,De tre saksøkte er i familie med D; C er hans ...,"[0.9660213589668274, -0.2151397466659546, 1.25..."
3,tagd-2021-84238_1_1,[1],[1],3,0,De saksøkte har hatt formelle roller i flere a...,"[-0.6119834184646606, 0.1516796201467514, 1.15..."
4,tagd-2021-84238_2_0,[2],[0],1,2,Selskapene de var delaktige i skulle ha aktivi...,"[0.7227107882499695, -1.1757646799087524, 1.02..."


In [4]:
sample = df.sample(1)
sample_text = sample["sent_text"].values[0]
sample_emb = sample["sbert_768"].values[0]

In [6]:
from semantic_match import get_matches

In [7]:
def print_matches(matches):
    for matcher in matches.values():
        print(f"📚: {matcher.id}")
        # sort paragraphs:
        paragraphs = dict(sorted(matcher.paragraphs.items()))
        for p_id, paragraph in paragraphs.items():
            print(f"📑 Paragraf {p_id} har {len(paragraph.sentences)} treff")
            para_sents = paragraph.sentences
            # sort based on whether "is_related" is True or False
            sorted_sents = sorted(para_sents, key=lambda x: 1 if x["is_related"] else 0)
            for sentence in sorted_sents:
                dybde = 1 if sentence["is_related"] else 0
              
              
                # tab spaces based on depth:
                tabs = "\t" * (dybde)
                # print(f"{tabs}Nivå {dybde}, Setning {sentence['id']}\n{tabs}📄'{sentence['text']}'")
                print(f"Setning {sentence['id']}\n{tabs}📄'{sentence['text']}'")
            print()

In [8]:
query = "uforsvarlig bruk av sykkel"
embedding_matches = get_matches(sbert, index, df, query, k=10)
print_matches(embedding_matches)

📚: tagd-2021-84238
📑 Paragraf 6 har 2 treff
Setning 4
📄'i 2007.'
Setning 0
	📄'Varsel om endring av ligning og om tilleggsskatt for inntektsårene 2004 – 2007 ble sendt til ['

📑 Paragraf 30 har 2 treff
Setning 5
	📄'Det betyr at ingen del av erstatningskravene er foreldet.'
Setning 0
	📄'I denne saken hadde staten ikke nødvendig kunnskap om erstatningskravene før bokettersynsrapporter var utarbeidet, første gang 9. oktober 2017 ([virksomhet4] AS).'

📑 Paragraf 32 har 1 treff
Setning 1
	📄'De saksøkte har uansett fullstendig unnlatt å oppfylle sine plikter som daglig leder, styreleder og styremedlem.'

📑 Paragraf 48 har 2 treff
Setning 5
	📄'Det betyr at ingen del av erstatningskravene er foreldet.'
Setning 0
	📄'I denne saken hadde staten ikke nødvendig kunnskap om erstatningskravene før bokettersynsrapporter var utarbeidet, første gang 9. oktober 2017 ([virksomhet4] AS).'

📑 Paragraf 50 har 1 treff
Setning 1
	📄'De saksøkte har uansett fullstendig unnlatt å oppfylle sine plikter som dagli